# Getting started with `pandas`

`pandas` is the python library for dealing with data that is organised as a "spreadsheet".

It is incredibly powerful, and allows for very terse code.

### Pros
* It’s terse, so you can develop quickly

### Cons
* It’s terse, so it tends to be more write-only code than not

![Power tools ahead](http://49.media.tumblr.com/tumblr_lvck9qVHou1qbvl2io1_400.gif)

In [ ]:
%matplotlib inline

In [ ]:
# pandas is a library for handling spreadsheets
import pandas
import matplotlib.pyplot as plt

In [ ]:
# create a DataFrame, this is what pandas calls spreadsheets
# Let's use our cleaned up ebola data
df = pandas.read_csv("ebola-edited.csv")

In [ ]:
# checkout five random entries/rows
df.sample(5)

In [ ]:
# all values in the Value column
df.Value

In [ ]:
# Can easily compute statistics on a column
df.Value.mean() # also: min(), max(), std()

In [ ]:
# or examine unique values and their count
df.Sources.value_counts()

# Let's start analysing

First, some basic sanity checks. Over what range of dates do we have data?

In [ ]:
df.Date.min(), df.Date.max()

In [ ]:
# Actually, it would be useful to have dates as dates not just strings
# this turns dates into "Timestamps" which means python can be smarter about
# what we mean when we perform operations on the dates
df = pandas.read_csv("ebola-edited.csv", parse_dates=[4])

In [ ]:
df.sample(5)
# no spurious hours and minutes anymore as these are dates

In [ ]:
# Why do we have data back to 1900??
df.Date.min(), df.Date.max()

In [ ]:
# what entries were made in 1900???
df[(df.Date == df.Date.min())]

# Data entry mistake

Someone entered data for 1900, this must be a mistake.

Googling for "Sitrep 289 archive" I found: http://www.who.int/csr/disease/ebola/situation-reports/archive/en/

Not situation report for January 29th, only one from January 28th 2015. Let's assume
this is the one. What other entries were made for that day referencing
the same Sitrep?

In [ ]:
# entries on Jan 29 or the weird date and linked to Sitrep 289
sub_df = df[((df.Date == "2015-01-29") | (df.Date == "1900-01-04"))& (df.Link == "Sitrep 289 29Jan")]

In [ ]:
sub_df[sub_df.Localite == "National"]

In [ ]:
# Looks like our hunch was right, let's replace all 1900-01-04 dates with Jan 29 2015
df2 = df.replace(to_replace={"Date": {pandas.Timestamp("1900-01-04"): pandas.Timestamp("2015-01-29")}})
# check the minimum date makes sense now
df2.Date.min()

# Take a closer look at each area of Guinea

How did the number of cumulative deaths evolve over time for some of the
regions of Guinea?

In [ ]:
# select entries counting Deaths from Guinea
guinea = df2[((df2.Country == "Guinea") & (df2.Category == "Deaths"))]

# something to investigate later, why are their duplicate entries?
guinea_no_dupes = guinea.drop_duplicates(['Date', 'Localite'])
print("duplicate entries:", guinea.shape, guinea_no_dupes.shape)

## Pivot tables

Right now have a table of observations, really want a time series for each area
`pivot(`) creates a [Pivot Table](https://en.wikipedia.org/wiki/Pivot_table).
The new dataframe will have `Date` as index,
one column per unique `Localite` and use the values from the `Value` column as values.

In [ ]:
guinea_no_dupes.pivot?

In [ ]:
# now for real
guinea_ts = guinea_no_dupes.pivot(index='Date', columns='Localite', values='Value')

In [ ]:
guinea_ts.describe().T

In [ ]:
# remove the nation wide numbers
guinea_ts_local = guinea_ts.drop('National', axis=1)
guinea_ts_local.describe().T

In [ ]:
guinea_subset_ts = guinea_ts_local[['Conakry', 'Coyah', 'Nzerekore', 'Macenta']]
guinea_subset_ts.plot()

In [ ]:
# can also split into several subplots
guinea_subset_ts.plot(subplots=True)

In [ ]:
# Or look at the rate of nation wide cases per day
guinea_national_rate = guinea_ts['National'].diff()

fig, (ax1,ax2) = plt.subplots(1, 2, sharex=True, figsize=(12,6))
guinea_national_rate.plot(ax=ax1)
guinea_ts['National'].plot(ax=ax2)
# No plot is complete without labeling!!
ax1.set_ylabel("Cases per day")
ax2.set_ylabel("Cumulative cases")
fig.suptitle("Nation wide numbers for Guinea", size=16)

In [ ]:
# store our reduced Guinea dataset in a CSV file
# Can easily be imported by Excel or used for
# making some map based visualisations
guinea_ts.to_csv("guinea-ebola.csv")